In [ ]:
import numpy as np
from numba import njit, prange
import csv
import os
import pandas as pd
from glob import glob
import time

In [ ]:
# Import all files (cell zones) under one experiment name
# Imported file names are printed

target_directory = '/Volumes/STORM_Nathalie/STORM DeMixing/210415 DEP647_VAMP2 8DIV/'
list_of_files_for_analysis = [file for file in glob(target_directory + '*/Demix/*.csv')]
#print(*list_of_files_for_analysis, sep="\n")

In [ ]:
#function calc_distances_squared_two calculates the closest point from vec_2 to vec_1 in 3D space.

@njit(fastmath=True,parallel=True)
def calc_distance_squared_two(vec_1,vec_2):

    res=np.empty((vec_1.shape[0]),dtype=vec_1.dtype)
    for i in prange(vec_1.shape[0]):
        res[i] = np.min(np.sqrt((vec_1[i,0]-vec_2[:,0])**2+(vec_1[i,1]-vec_2[:,1])**2+(vec_1[i,2]-vec_2[:,2])**2))
        
    return res


In [ ]:
# This function calculates the minimum nearest neighbour distances
# for many cell zones in the experiment and saves them to a new
# csv. 

DEFAULT_OUT_DIR = "/Users/isabellegarnreiter/Documents/vesicleSTORM/results/Output/"

def analysis(list_of_files_for_analysis, save=True):
    Min_dist_dict = {}
    for count, file_name in enumerate(list_of_files_for_analysis):
        if '_w2_' in file_name:
            print("pass")
        else:
            vesicles = pd.read_csv(file_name)
            synapse = pd.read_csv(list_of_files_for_analysis[count+1])
            
            vesicles = file_1[['x [nm]', 'y [nm]', 'z [nm]']].to_numpy(dtype=np.float64)
            synapse_marker = file_2[['x [nm]', 'y [nm]', 'z [nm]']].to_numpy(dtype=np.float64)
            distances = calc_distance_squared_two(vesicles, synapse_marker)
            
            if __name__ == '__main__':
                assert distances[0] = np.min(np.sqrt((vesicles[0,0]-synapse_marker[:,0])**2+
                                                     (vesicles[0,1]-synapse_marker[:,1])**2+
                                                     (vesicles[0,2]-synapse_marker[:,2])**2))
            
            splits = (file_name).split('/')
            new_file_name = "{splits[4]}_{splits[5]}"
            
            Min_dist_dict[new_file_name] = distances
            
            
            
            if save==True
                file_1['min dist (um)'] = distances
                file_1.to_csv(os.path.join(out_directory, f"{new_file_name}.csv"))
            
            print("Processed " + str(new_file_name(file_name)))
            

In [ ]:
list_of_sample_min_dists = analysis(list_of_files_for_analysis[1:3], save==False)
